In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [2]:
nltk.download("stopwords")
STOPWORDS = stopwords.words("english")
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alokrajgupta/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import pandas as pd

df = pd.read_csv(r"../data/labeled_projects.csv")
df

id           created_on  \
0       6  2020-02-20 06:43:18   
1       7  2020-02-20 06:47:21   
2       9  2020-02-24 16:24:45   
3      15  2020-02-28 23:55:26   
4      19  2020-03-03 13:54:31   
..    ...                  ...   
950  2431  2020-10-05 12:36:02   
951  2432  2020-10-05 13:09:58   
952  2434  2020-10-06 02:13:01   
953  2436  2020-10-06 13:29:44   
954  2437  2020-10-06 14:53:39   

                                                 title  \
0    Comparison between YOLO and RCNN on real world...   
1    Show, Infer & Tell: Contextual Inference for C...   
2                         Awesome Graph Classification   
3                      Awesome Monte Carlo Tree Search   
4                                  Diffusion to Vector   
..                                                 ...   
950    A Survey of the State of Explainable AI for NLP   
951                           Topic Modeling with BERT   
952                          OpenMMLab Computer Vision   
953    Machine Learning Methods Explained (+ Examples)   
954                                             Rasoee   

                                           description  \
0    Bringing theory to experiment is cool. We can ...   
1    The beauty of the work lies in the way it arch...   
2    A collection of important graph embedding, cla...   
3    A curated list of Monte Carlo tree search pape...   
4    Reference implementation of Diffusion2Vec (Com...   
..                                                 ...   
950  Overview of the operations and explainability ...   
951  Leveraging 🤗 Transformers and a class-based TF...   
952  MMCV is a python library for CV research and s...   
953  Most common techniques used in data science pr...   
954  A powerful web and mobile application that ide...   

                             tag  
0                computer-vision  
1                computer-vision  
2                 graph-learning  
3         reinforcement-learning  
4                 graph-learning  
..                           ...  
950  natural-language-processing  
951  natural-language-processing  
952              computer-vision  
953       reinforcement-learning  
954              computer-vision  

[955 rows x 5 columns]

In [6]:
# Input
df["text"] = df.title + " " + df.description

In [7]:
def clean_text(text, lower=True, stem=False, stopwords=STOPWORDS):
    """Clean raw text."""
    # Lower
    if lower:
        text = text.lower()

    # Remove stopwords
    if len(stopwords):
        pattern = re.compile(r'\b(' + r"|".join(stopwords) + r")\b\s*")
        text = pattern.sub('', text)

    # Spacing and filters
    text = re.sub(
        r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text
    )  # add spacing between objects to be filtered
    text = re.sub("[^A-Za-z0-9]+", " ", text)  # remove non alphanumeric chars
    text = re.sub(" +", " ", text)  # remove multiple spaces
    text = text.strip()  # strip white space at the ends

    # Remove links
    text = re.sub(r"http\S+", "", text)

    # Stemming
    if stem:
        text = " ".join([stemmer.stem(word, to_lowercase=lower) for word in text.split(" ")])

    return text


In [8]:
# Apply to dataframe
original_df = df.copy()
df.text = df.text.apply(clean_text, lower=True, stem=False)
print (f"{original_df.text.values[0]}\n{df.text.values[0]}")

Comparison between YOLO and RCNN on real world videos Bringing theory to experiment is cool. We can easily train models in colab and find the results in minutes.
comparison yolo rcnn real world videos bringing theory experiment cool easily train models colab find results minutes


In [9]:
import json

In [10]:
# Accepted tags (external constraint)
ACCEPTED_TAGS = ["natural-language-processing", "computer-vision", "mlops", "graph-learning"]

In [11]:
# Out of scope (OOS) tags
oos_tags = [item for item in df.tag.unique() if item not in ACCEPTED_TAGS]
oos_tags

['reinforcement-learning', 'time-series']

In [12]:
# Samples with OOS tags
oos_indices = df[df.tag.isin(oos_tags)].index
df[df.tag.isin(oos_tags)].head()

id           created_on                                       title  \
3    15  2020-02-28 23:55:26             Awesome Monte Carlo Tree Search   
37  121  2020-03-24 04:56:38  Deep Reinforcement Learning in TensorFlow2   
67  218  2020-04-06 11:29:57         Distributional RL using TensorFlow2   
74  239  2020-04-06 18:39:48               Prophet: Forecasting At Scale   
95  277  2020-04-07 00:30:33       Curriculum for Reinforcement Learning   

                                          description                     tag  \
3   A curated list of Monte Carlo tree search pape...  reinforcement-learning   
37  deep-rl-tf2 is a repository that implements a ...  reinforcement-learning   
67  🐳 Implementation of various Distributional Rei...  reinforcement-learning   
74  Tool for producing high quality forecasts for ...             time-series   
95  Curriculum learning applied to reinforcement l...  reinforcement-learning   

                                                 text  
3   awesome monte carlo tree search curated list m...  
37  deep reinforcement learning tensorflow2 deep r...  
67  distributional rl using tensorflow2 implementa...  
74  prophet forecasting scale tool producing high ...  
95  curriculum reinforcement learning curriculum l...

In [15]:
df['tag'] = df['tag'].apply(lambda x: "other" if x in oos_tags else x)

In [16]:
df['tag'].value_counts()

natural-language-processing    388
computer-vision                356
other                           87
mlops                           79
graph-learning                  45
Name: tag, dtype: int64

In [18]:
# Minimum frequency required for a tag
from collections import Counter
min_freq = 75
tags = Counter(df.tag.values)

In [19]:
tags

Counter({'computer-vision': 356,
         'graph-learning': 45,
         'other': 87,
         'natural-language-processing': 388,
         'mlops': 79})

In [22]:
def filter(tag, include=[]):
    """Determine if a given tag is to be included."""
    if tag not in include:
        tag = None
    return tag


In [23]:
# Filter tags that have fewer than <min_freq> occurrences
tags_above_freq = Counter(tag for tag in tags.elements()
                          if (tags[tag] >= min_freq))
df.tag = df.tag.apply(filter, include=list(tags_above_freq.keys()))

In [24]:
# Fill None with other
df.tag = df.tag.fillna("other")

In [25]:
import numpy as np
import random

In [26]:
# Get data
X = df.text.to_numpy()
y = df.tag

In [27]:
class LabelEncoder(object):
    """Encode labels into unique indices"""
    def __init__(self, class_to_index={}):
        self.class_to_index = class_to_index or {}  # mutable defaults ;)
        self.index_to_class = {v: k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())

    def __len__(self):
        return len(self.class_to_index)

    def __str__(self):
        return f"<LabelEncoder(num_classes={len(self)})>"

    def fit(self, y):
        classes = np.unique(y)
        for i, class_ in enumerate(classes):
            self.class_to_index[class_] = i
        self.index_to_class = {v: k for k, v in self.class_to_index.items()}
        self.classes = list(self.class_to_index.keys())
        return self

    def encode(self, y):
        encoded = np.zeros((len(y)), dtype=int)
        for i, item in enumerate(y):
            encoded[i] = self.class_to_index[item]
        return encoded

    def decode(self, y):
        classes = []
        for i, item in enumerate(y):
            classes.append(self.index_to_class[item])
        return classes

    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {"class_to_index": self.class_to_index}
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)


In [28]:
# Encode
label_encoder = LabelEncoder()
label_encoder.fit(y)
num_classes = len(label_encoder)

In [29]:
label_encoder.class_to_index

{'computer-vision': 0,
 'mlops': 1,
 'natural-language-processing': 2,
 'other': 3}

In [30]:
label_encoder.index_to_class

{0: 'computer-vision',
 1: 'mlops',
 2: 'natural-language-processing',
 3: 'other'}

In [31]:
# Encode
label_encoder.encode(["computer-vision", "mlops", "mlops"])

array([0, 1, 1])

In [32]:
# Decode
label_encoder.decode(np.array([0, 1, 1]))

['computer-vision', 'mlops', 'mlops']

In [33]:
# Encode all our labels
y = label_encoder.encode(y)
print (y.shape)

(955,)


In [34]:
from sklearn.model_selection import train_test_split

In [35]:
# Split sizes
train_size = 0.7
val_size = 0.15
test_size = 0.15

In [36]:
# Split (train)
X_train, X_, y_train, y_ = train_test_split(
    X, y, train_size=train_size, stratify=y)

In [37]:
print (f"train: {len(X_train)} ({(len(X_train) / len(X)):.2f})\n"
       f"remaining: {len(X_)} ({(len(X_) / len(X)):.2f})")

train: 668 (0.70)
remaining: 287 (0.30)


In [38]:
# Split (test)
X_val, X_test, y_val, y_test = train_test_split(
    X_, y_, train_size=0.5, stratify=y_)

In [39]:
print(f"train: {len(X_train)} ({len(X_train)/len(X):.2f})\n"
      f"val: {len(X_val)} ({len(X_val)/len(X):.2f})\n"
      f"test: {len(X_test)} ({len(X_test)/len(X):.2f})")

train: 668 (0.70)
val: 143 (0.15)
test: 144 (0.15)


In [40]:
# Get counts for each class
counts = {}
counts["train_counts"] = {tag: label_encoder.decode(y_train).count(tag) for tag in label_encoder.classes}
counts["val_counts"] = {tag: label_encoder.decode(y_val).count(tag) for tag in label_encoder.classes}
counts["test_counts"] = {tag: label_encoder.decode(y_test).count(tag) for tag in label_encoder.classes}

In [41]:
# View distributions
pd.DataFrame({
    "train": counts["train_counts"],
    "val": counts["val_counts"],
    "test": counts["test_counts"]
}).T.fillna(0)

computer-vision  mlops  natural-language-processing  other
train              249     55                          272     92
val                 53     12                           58     20
test                54     12                           58     20

In [42]:
# Adjust counts across splits
for k in counts["val_counts"].keys():
    counts["val_counts"][k] = int(counts["val_counts"][k] * \
        (train_size/val_size))
for k in counts["test_counts"].keys():
    counts["test_counts"][k] = int(counts["test_counts"][k] * \
        (train_size/test_size))

In [43]:
dist_df = pd.DataFrame({
    "train": counts["train_counts"],
    "val": counts["val_counts"],
    "test": counts["test_counts"]
}).T.fillna(0)
dist_df

computer-vision  mlops  natural-language-processing  other
train              249     55                          272     92
val                247     56                          270     93
test               252     56                          270     93

In [44]:
# Split DataFrames
train_df = pd.DataFrame({"text": X_train, "tag": label_encoder.decode(y_train)})
val_df = pd.DataFrame({"text": X_val, "tag": label_encoder.decode(y_val)})
test_df = pd.DataFrame({"text": X_test, "tag": label_encoder.decode(y_test)})
train_df.head()

text  \
0  bottleneck graph neural networks implications ...   
1  anomaly detection keras tensorflow deep learni...   
2  bert loses patience fast robust inference earl...   
3  tempering expectations gpt 3 openai api closer...   
4  indian paper currency prediction trained model...   

                           tag  
0                        other  
1              computer-vision  
2  natural-language-processing  
3  natural-language-processing  
4              computer-vision